## Задание 1

Опеределите части речи слов в этом тексте, используя пайморфи. В форму запишите слова, которым приписан тэг VERB. 

In [ ]:
text = "Зонды будут запущены в космос в 2029 году и отправятся ко второй точке Лагранжа в системе Солнце"

In [ ]:
from pymorphy2 import MorphAnalyzer
morph = MorphAnalyzer()

In [ ]:
for w in text.split():
    p = morph.parse(w)[0]
    print(f'{w}:{p.tag.POS}')

Зонды:NOUN
будут:VERB
запущены:PRTS
в:PREP
космос:NOUN
в:PREP
2029:None
году:NOUN
и:CONJ
отправятся:VERB
ко:PREP
второй:ADJF
точке:NOUN
Лагранжа:NOUN
в:PREP
системе:NOUN
Солнце:NOUN


## Задание 2

На данных предложениях обучите TFIDF векторайзер из sklearn (с дефолтными параметрами). Найдите слово с самым высоким коэффициентом tfidf в первом тексте. Вставьте его в форму.

In [ ]:
import requests
from sklearn.feature_extraction.text import TfidfVectorizer

r = requests.get(
    'https://raw.githubusercontent.com/mannefedov/compling_nlp_hse_course/master/data/anna_karenina.txt'
)

# работайте с этими предложениями
sentences = r.text.split('\n')
sentences = [sent for sent in sentences if len(sent) > 10]

In [ ]:
tf = TfidfVectorizer()
tfidf_matrix = tf.fit_transform(sentences)

In [ ]:
feature_names = tf.get_feature_names()

In [ ]:
doc = 0 # берем первый текст
feature_index = tfidf_matrix[doc,:].nonzero()[1]
tfidf_scores = zip(feature_index, [tfidf_matrix[doc, x] for x in feature_index])

my_dict = {}
for w, s in [(feature_names[i], s) for (i, s) in tfidf_scores]:
    my_dict[w] = s

In [ ]:
import operator
sorted_d = dict( sorted(my_dict.items(), key=operator.itemgetter(1),reverse=True))
sorted_d

{'вырождается': 0.19671514407338786,
 'устои': 0.19671514407338786,
 'ослабевают': 0.19671514407338786,
 'нравы': 0.19671514407338786,
 'падают': 0.19671514407338786,
 'буржуазного': 0.19671514407338786,
 'натиском': 0.19671514407338786,
 'уклада': 0.19671514407338786,
 'патриархального': 0.19671514407338786,
 'рушатся': 0.19671514407338786,
 'страницах': 0.19671514407338786,
 'приметами': 0.19671514407338786,
 'насыщенное': 0.19671514407338786,
 'остропроблемное': 0.19671514407338786,
 'психологически': 0.19671514407338786,
 'аристократия': 0.18807778290532548,
 'прогресса': 0.18807778290532548,
 'произведения': 0.18807778290532548,
 'утонченное': 0.18807778290532548,
 'остатки': 0.1819494776291046,
 'произведение': 0.1819494776291046,
 'сложное': 0.1819494776291046,
 'толстой': 0.17331211646104222,
 'показывает': 0.1700283441478517,
 'семейные': 0.16243032831667098,
 'россии': 0.13832880029899972,
 'каренина': 0.1318596500912228,
 'времени': 0.12128437592213807,
 'как': 0.10254114162

## Задание 3

Токенизируйте текст Анны Карениной с помощью библиотеки раздел, привидите все к нижнему регистру. Рассчитайте статистики биграммов и униграммов. Найдите наиболее вероятное продолжение "красный ...". Вставьте наиболее веротяное следующее слово в форму.

In [ ]:
! pip install razdel
from razdel import tokenize
import gensim
from collections import Counter

In [32]:
def tokenize_via_razdel(texts):
  tokenized = []
  for t in texts:
    tokens = list(tokenize(t))
    tokens = [_.text for _ in tokens]
    tokenized.extend(tokens)
  return tokenized

In [33]:
sent_tokenized = tokenize_via_razdel(sentences)

In [34]:
vocab = Counter(sent_tokenized)

In [36]:
# ищем кол-во вхождений каждой n-граммы
def ngrammer(tokens, n):
    ngrams = []
    # проходимся по токенам
    for i in range(0,len(tokens)-n+1):
        ngrams.append(' '.join(tokens[i:i+n]))
    return ngrams

In [41]:
unigrams = vocab
bigrams = Counter(ngrammer(sent_tokenized,2))

In [42]:
probas_uni = Counter({word:c/len(sent_tokenized) for word, c in unigrams.items()})
probas_bi = Counter({word:c/len(sent_tokenized) for word, c in bigrams.items()})

In [48]:
probas_bi = dict(probas_bi)

In [54]:
res = [ v for k,v in probas_bi.items() if k.startswith('красный')]

In [57]:
max(res)

1.447995250575578e-05

In [52]:
res = [ [k,v] for k,v in probas_bi.items() if k.startswith('красный')]

In [53]:
res

[['красный мешочек', 1.447995250575578e-05],
 ['красный огонь', 2.895990501151156e-06],
 ['красный ,', 5.791981002302312e-06],
 ['красный платок', 2.895990501151156e-06],
 ['красный .', 2.895990501151156e-06]]

## Задание 4

Расчитайте ненормализованное растояние Дамерау-Левенштейна и ненормализованное растояние Левенштейна между этими словами (каждое с каждым), найдите пару слов, для которых эти расстояния отличаются. Вставьте эти слова в форму

In [ ]:
words = ["решение","ршеение","ренешик","рещиние","ришение"]
import itertools
pairs = [list(p) for p in itertools.product(words, repeat=2)]

In [ ]:
import textdistance

res1 = []
for p in pairs:
    res1 = textdistance.damerau_levenshtein(p[0], p[1])
    res2 = textdistance.levenshtein(p[0], p[1])
    if res1 != res2:
        print(p)

['решение', 'ршеение']
['ршеение', 'решение']


## Задание 5

Загрузите токенизатор distilbert-base-uncased из huggingface. Токенизируйте предложения из Анны Карениной с помощью предобученного токенизатора и обучите Fastext модель со следующими параметрами: cbow, размер вектора 300, размер окна 5, минимальный размер символьного нграмма - 2, максимальный размер символьного нграмма - 7. Найдите ближайшее слово к слову "каренина", вставьте его в форму.

In [ ]:
# ! pip install transformers
import tensorflow as tf
from transformers import TFAutoModel
from transformers import AutoTokenizer

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/483 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/455k [00:00<?, ?B/s]

In [ ]:
sents_tokenized_bert = []

for sent in sentences:
    tokens = tokenizer.tokenize(sent)
    sents_tokenized_bert.append(tokens)

In [ ]:
from gensim.models import FastText

In [ ]:
ft = FastText(sents_tokenized_bert, size=300, window=5, min_n=2, max_n=7, sg=0)

In [ ]:
ft.wv.most_similar('каренина')

[('##на', 0.6452010869979858),
 ('##ка', 0.6099033951759338),
 (')', 0.4820539057254791),
 ('landau', 0.3349595069885254),
 ('–', 0.2926204204559326),
 ('##ович', 0.270717978477478),
 ('right', 0.27019062638282776),
 ('pardon', 0.23741167783737183),
 ('cher', 0.23715996742248535),
 ('»', 0.20935595035552979)]